In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import timedelta, date
import matplotlib.pyplot as plt
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import pyspark.sql.functions as psf
from urllib.parse import unquote, urlparse, parse_qs

# Parameters section:

In [2]:
DESTINATION_FOLDER = "how_we_read_wikipedia"

start_date = date(2021, 1, 3)
end_date = date(2021, 2, 1)
SNAPSHOT = '2021-01'

# required to iterate the range of dates
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Generate random salt applied both to IP and user-agent

The salt strings are never printed 

In [3]:
import string
import random

def get_random_string(size):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(size))

salt1 = get_random_string(16)
salt2 = get_random_string(16)

Example of anonymized user identifier:

In [4]:
import hashlib

# Different salt. The secret one is never printed
s1 = get_random_string(16)
s2 = get_random_string(16)
print("Example of user identifier (different salt):\n")
print("md5(<IP>{}<ACCESS_METHOD>) + md5(<USER_AGENT>{}) = ".format(s1, s2))
print("\t"+hashlib.md5("<IP>{}<ACCESS_METHOD>".format(s1).encode()).hexdigest()+
      hashlib.md5("<USER_AGENT>{}".format(s2).encode()).hexdigest())

Example of user identifier (different salt):

md5(<IP>BIZ4QS0VQBO75NKM<ACCESS_METHOD>) + md5(<USER_AGENT>E8N2XB146ZHO9YOQ) = 
	4e5e6498f29a8f34a8bbb88021a3d5458332b574bf83332111aec3f20146a31b


# Get countries with more that 500 views every day

In [5]:
query = """
    select geocoded_data['country_code'] country_code
    from wmf.webrequest 
    where day = {}
    AND month = {}
    AND year = {}
    AND x_analytics_map['loggedIn'] is NULL
    AND (namespace_id = 0 OR namespace_id = 4 OR namespace_id = 100)
    AND agent_type = 'user'
    AND is_pageview = TRUE
    AND (uri_host = 'en.wikipedia.org' or uri_host = 'en.m.wikipedia.org')
    AND http_method= 'GET'
    AND ts is not NULL
"""

countries = set()
for day in daterange(start_date, end_date):
    # Load the logs of the day
    pageview_logs = spark.sql(query.format(day.day, day.month, day.year)).repartition(30)
    # Group by country
    pageviews_by_country = pageview_logs.groupBy("country_code").agg(count("*").alias("total"))
    # Keep only countries with 500 views in this day
    pageviews_by_country_filtered = pageviews_by_country.where("total>=500")
    # Collect the countries names
    countries500views = pageviews_by_country_filtered.select("country_code").collect()
    # Get the intersection with the other days
    if len(countries) < 1:
        countries = set(countries500views)
    else:
        countries = countries.intersection(countries500views)

List of the countries included:

In [6]:
print([c.country_code for c in countries500views])
len(countries)

['LT', 'MM', 'DZ', 'CI', 'TC', 'FI', 'AZ', 'SC', 'UA', 'RO', 'ZM', 'SL', 'NL', 'SB', 'LA', 'BS', 'MN', 'BW', 'PL', 'AM', 'PS', 'RE', 'MK', 'MX', 'PF', 'GL', 'EE', 'VG', 'SM', 'CN', 'AT', 'IQ', 'RU', 'NA', 'CG', 'AD', 'HR', 'SV', 'LI', 'CZ', 'NP', 'PT', 'SO', 'GG', 'PG', 'KY', 'GH', 'HK', 'CV', 'BN', 'LR', 'TW', 'BD', 'PY', 'LB', 'CL', 'TO', 'LY', 'ID', 'FK', 'SA', 'PK', 'AU', 'CA', 'BM', 'MW', 'BL', 'UZ', 'NE', 'GB', 'MT', 'YE', 'BR', 'KZ', 'BY', 'HN', 'NC', 'MD', 'GT', 'DE', 'AW', 'GN', 'ES', 'EC', 'IR', 'BH', 'MO', 'VI', 'IL', 'VE', 'TR', 'ME', 'MR', 'ZA', 'CR', 'SX', 'AI', 'GU', 'KR', 'TZ', 'US', 'RS', 'AL', 'MY', 'JM', 'IN', 'CK', 'LC', 'GM', 'AE', 'MQ', 'CM', 'RW', 'TG', 'FR', 'GF', 'CH', 'MG', 'TN', 'GQ', 'TL', 'GR', 'PA', 'GI', 'TD', 'SD', 'AG', 'DJ', 'MC', 'JO', 'BA', 'ET', 'SG', 'BF', 'CU', 'IT', 'GW', 'MV', 'FO', 'SE', 'BG', 'PH', 'FJ', 'WS', 'GE', 'SK', 'CW', 'MH', 'FM', 'LV', 'PE', 'MU', 'LS', 'GD', 'MZ', 'DM', 'DO', 'QA', 'XK', 'BZ', 'TH', 'EG', 'BI', 'BJ', 'MF', 'GY', 'JP

219

In [7]:
countries_whitelist = spark.createDataFrame(sc.parallelize(list(countries))).cache()
countries_whitelist.show(5)

+------------+
|country_code|
+------------+
|          SE|
|          CL|
|          AZ|
|          JE|
|          SG|
+------------+
only showing top 5 rows



# Get the users with edits

Get all the users indentifiers with an edit. Get the `user_identifier` and the day/month localised with the local timezone. This rows will left joined later as a blacklist to exclude all views of that user in the day of the edit.

In [8]:
query = """
select CONCAT(MD5(CONCAT(CONCAT(client_ip, '{}'), access_method)), MD5(CONCAT(user_agent, '{}'))) as user_identifier,
        from_utc_timestamp(ts, geocoded_data['timezone']) as local_time
from wmf.webrequest 
where day = {}
AND month = {}
AND year = {}
AND agent_type = 'user'
AND http_method= 'GET'
AND ts is not NULL
AND (uri_host = 'en.wikipedia.org' or uri_host = 'en.m.wikipedia.org')
AND (uri_query LIKE '%action=edit%' OR uri_query LIKE '%action=visualeditor%' 
    OR uri_query LIKE '%&intestactions=edit&intestactionsdetail=full&uiprop=options%')
"""

edits_rdd = sc.emptyRDD()
for day in daterange(start_date, end_date):
    daily_edits_rdd = spark.sql(query.format(salt1, salt2, day.day, day.month, day.year))\
                .selectExpr("user_identifier", "day(local_time) day", "month(local_time) month")\
                .distinct().repartition(30).rdd
    edits_rdd = edits_rdd.union(daily_edits_rdd)
    
editors = spark.createDataFrame(edits_rdd).distinct().repartition(200).cache()

print("Total editors-day:", editors.count())
print("Total editors:", editors.select("user_identifier").distinct().count())
editors

Total editors-day: 7063635
Total editors: 6181225


DataFrame[user_identifier: string, day: bigint, month: bigint]

-----------

### Function to drop sensitive information

In [9]:
editors.registerTempTable("editors")

def without_sensitive_data(current_dataframe):
    # Add the day and month used in the join with the editors
    current_dataframe = current_dataframe.selectExpr("*", "day(local_time) day", "month(local_time) month")
    current_dataframe.createOrReplaceTempView("current_dataframe")
    # LEFT JOIN on identifier and day/month. Keep non-matches to exclude editors
    query_no_editors = """
        SELECT c.*
        FROM current_dataframe c
        LEFT JOIN editors e
        ON c.user_identifier = e.user_identifier
        AND c.day = e.day
        AND c.month = e.month
        WHERE e.user_identifier IS NULL
    """
    clean_dataframe = spark.sql(query_no_editors).drop("day").drop("month")
    # Join the dataframe with the whitelisted countries
    return clean_dataframe.join(countries_whitelist, "country_code")

----------

# Get page views

Let's start with the redirects:

In [12]:
# Select the redirect pages from the snapshot
redirection_mapping = spark.sql("""
SELECT rd_from page_id, rd_title redirect_to
FROM wmf_raw.mediawiki_redirect
WHERE wiki_db = 'enwiki'
AND snapshot = '{}'
AND (rd_namespace = 0 OR rd_namespace = 4 OR rd_namespace = 100)
""".format(SNAPSHOT))

# Get the page title of the redirect
# It is needed for the join with the pageloads
redirection_mapping.createOrReplaceTempView("redirection_mapping")
title_to_title = spark.sql("""
SELECT mp.page_id page_id_from, mp.page_title page_title_from, rm.redirect_to page_title_to
FROM wmf_raw.mediawiki_page mp
JOIN redirection_mapping rm
ON mp.page_id = rm.page_id
WHERE mp.wiki_db = 'enwiki'
AND mp.snapshot = '{}'
AND (mp.page_namespace = 0 OR mp.page_namespace = 4 OR mp.page_namespace = 100)
""".format(SNAPSHOT)).cache()

title_to_title.createOrReplaceTempView("title_to_title")

Get the pageviews. There is not need to anonymize the IP in this stage. We will drop it later.

In [13]:
# User must be not loggedIn, not a bot and the view must be in the English version of Wikipedia
query = """
select CONCAT(MD5(CONCAT(CONCAT(client_ip, '{}'), access_method)), MD5(CONCAT(user_agent, '{}'))) as user_identifier,
        geocoded_data['country_code'] country_code,
        http_status, referer,
        from_utc_timestamp(ts, geocoded_data['timezone']) as local_time,
        access_method,
        pageview_info['page_title'] as page_title, page_id
from wmf.webrequest 
where day = {}
AND month = {}
AND year = {}
AND x_analytics_map['loggedIn'] is NULL
AND (namespace_id = 0 OR namespace_id = 4 OR namespace_id = 100)
AND agent_type = 'user'
AND is_pageview = TRUE
AND (uri_host = 'en.wikipedia.org' or uri_host = 'en.m.wikipedia.org')
AND http_method= 'GET'
AND ts is not NULL
"""

# Repeat for each day
for day in daterange(start_date, end_date):
    pageview_logs = spark.sql(query.format(salt1, salt2, day.day, day.month, day.year)).repartition(30)
    pageview_logs.createOrReplaceTempView("pageview_logs")
    
    # Add resolved redirects (add the day and month for later join)
    query_join_redirects = """
        SELECT l.*, 
            
            CASE WHEN tt.page_title_to is NULL 
                THEN l.page_title 
                ELSE tt.page_title_to 
            END as actual_destination
        FROM pageview_logs l
        LEFT JOIN title_to_title tt
        ON l.page_title = tt.page_title_from
    """
    logs_redirect_resolved = spark.sql(query_join_redirects).cache()
    
    # Remove sensitive data
    logs_filtered = without_sensitive_data(logs_redirect_resolved).cache()

    print("{}-{:02d}-{:02d} Country & editors filter keeps {}%".format(day.year, day.month, day.day, 
                                     100*logs_filtered.count() / logs_redirect_resolved.count()))
    # Write the pageviews of this day
    logs_filtered.write.mode("overwrite")\
                .parquet("{}/pageloads/{}{:02d}{:02d}".format(DESTINATION_FOLDER, day.year, day.month, day.day))
    print("{}-{:02d}-{:02d} DONE".format(day.year, day.month, day.day))
    
    # Drop cached dataframes
    logs_redirect_resolved.unpersist()
    logs_filtered.unpersist()
    
print("Format:")
logs_filtered

2021-01-03 Country & editors filter keeps 97.66262710735279%
2021-01-03 DONE
2021-01-04 Country & editors filter keeps 97.23297041781927%
2021-01-04 DONE
2021-01-05 Country & editors filter keeps 96.65064891005593%
2021-01-05 DONE
2021-01-06 Country & editors filter keeps 97.08144648273644%
2021-01-06 DONE
2021-01-07 Country & editors filter keeps 96.9810369401506%
2021-01-07 DONE
2021-01-08 Country & editors filter keeps 97.1275888595591%
2021-01-08 DONE
2021-01-09 Country & editors filter keeps 97.66890696906167%
2021-01-09 DONE
2021-01-10 Country & editors filter keeps 97.50091142766524%
2021-01-10 DONE
2021-01-11 Country & editors filter keeps 96.81622148942999%
2021-01-11 DONE
2021-01-12 Country & editors filter keeps 96.84827740993283%
2021-01-12 DONE
2021-01-13 Country & editors filter keeps 97.11226632944532%
2021-01-13 DONE
2021-01-14 Country & editors filter keeps 96.85168497880122%
2021-01-14 DONE
2021-01-15 Country & editors filter keeps 96.9090164561678%
2021-01-15 DONE
20

DataFrame[country_code: string, user_identifier: string, http_status: string, referer: string, local_time: timestamp, access_method: string, page_title: string, page_id: bigint, actual_destination: string]

# Get page previews

In [14]:
get_page_udf = udf(lambda uri_path: unquote(uri_path[uri_path.rfind('/')+1:]), StringType())

# Consider all previews. Note:
#    year year_original, month month_original, day day_original, 
#    hour hour_original, client_ip, user_agent_map: are required for the join with the long previews
# We'll discard them later.
preview_query = """
select CONCAT(MD5(CONCAT(CONCAT(client_ip, '{}'), access_method)), MD5(CONCAT(user_agent, '{}'))) as user_identifier,
        http_status, referer,
        from_utc_timestamp(ts, geocoded_data['timezone']) as local_time,
        uri_path,
        geocoded_data['country_code'] country_code,
        
        year year_original, month month_original, day day_original, hour hour_original, client_ip, user_agent_map
from wmf.webrequest 
where day = {}
AND month = {}
AND year = {}
AND x_analytics_map['loggedIn'] is NULL
AND agent_type = 'user'
AND http_method= 'GET'
AND ts is not NULL
AND (uri_host = 'en.wikipedia.org' or uri_host = 'en.m.wikipedia.org')
AND uri_path LIKE '/api/rest_v1/page/summary/%'
AND referer LIKE '%wikipedia.org/wiki%'
"""

# Consider the previews with at least 1 second
long_previews_query = """
select ip, 
    year, month, day, hour,
    useragent.browser_family,
    useragent.browser_major,
    useragent.os_family,
    useragent.device_family,
    event.source_title, event.page_title preview_title
from event.virtualpageview
where year = {}
and month = {}
and day = {}
and webhost = 'en.wikipedia.org'
and useragent.is_bot = false
"""

for day in daterange(start_date, end_date):
    # compile the query with parameters and repartition the data
    logs = spark.sql(preview_query.format(salt1, salt2, day.day, day.month, day.year)).repartition(30)
    
    # Extract the titles from the urls and drop the original fields
    logs_cleaned = logs.withColumn("preview_title", get_page_udf(logs.uri_path)).drop("uri_path")\
                    .withColumn("page_title", get_page_udf(logs.referer)).drop("referer")
    
    # Remove sensitive data
    previews_cleaned = without_sensitive_data(logs_cleaned).drop("country_code")
    
    # Get long previews
    long_previews = spark.sql(long_previews_query.format(day.year, day.month, day.day)).distinct()

    # Register the tables
    previews_cleaned.createOrReplaceTempView("previews_cleaned")
    long_previews.createOrReplaceTempView("long_previews")

    # LEFT JOIN previews_cleaned with the long previews to add a label TRUE or FALSE
    # Since the 2 tables have different schema, we have to maximise the matching by joining
    # on all the fields we can use. We add a boolean labels to mark if we found a long preview
    # from the same ip, on the same page, for the same link, in the the same hour, 
    # with the same browser/OS version
    query = """
        SELECT ap.*,
            CASE 
                WHEN lp.ip IS NULL THEN FALSE
                ELSE TRUE
            END as long_preview
        FROM previews_cleaned ap
        LEFT JOIN long_previews lp
        ON ap.client_ip = lp.ip
        AND ap.hour_original = lp.hour
        AND ap.day_original = lp.day
        AND ap.month_original = lp.month
        AND ap.year_original = lp.year
        AND ap.page_title = lp.source_title
        AND ap.preview_title = lp.preview_title
        AND ap.user_agent_map.browser_family = lp.browser_family
        AND ap.user_agent_map.browser_major = lp.browser_major
        AND ap.user_agent_map.os_family = lp.os_family
        AND ap.user_agent_map.device_family = lp.device_family
    """

    all_previews_labeled = spark.sql(query).drop("user_agent_map", "client_ip", "year_original", 
                                                 "month_original", "day_original", "hour_original")
    
    # Write the previews
    all_previews_labeled.write.mode("overwrite")\
                .parquet("{}/previews/{}{:02d}{:02d}".format(DESTINATION_FOLDER, day.year, day.month, day.day))
    print("{}-{:02d}-{:02d} DONE".format(day.year, day.month, day.day))
    
print("Format:")
all_previews_labeled

2021-01-03 DONE
2021-01-04 DONE
2021-01-05 DONE
2021-01-06 DONE
2021-01-07 DONE
2021-01-08 DONE
2021-01-09 DONE
2021-01-10 DONE
2021-01-11 DONE
2021-01-12 DONE
2021-01-13 DONE
2021-01-14 DONE
2021-01-15 DONE
2021-01-16 DONE
2021-01-17 DONE
2021-01-18 DONE
2021-01-19 DONE
2021-01-20 DONE
2021-01-21 DONE
2021-01-22 DONE
2021-01-23 DONE
2021-01-24 DONE
2021-01-25 DONE
2021-01-26 DONE
2021-01-27 DONE
2021-01-28 DONE
2021-01-29 DONE
2021-01-30 DONE
2021-01-31 DONE
Format:


DataFrame[user_identifier: string, http_status: string, local_time: timestamp, preview_title: string, page_title: string, long_preview: boolean]

-----
# Get clicks on images

In [15]:
# Get a row with the image name and title extracted from the urls
# Input format:
# DataFrame[user_identifier: string, local_time: timestamp, 
#           referer: string, uri_query: string, access_method: string]

def get_formatted_row(row):
    try:
        # Take the string after the last / and convert the http encoded text
        title = unquote(row.referer[row.referer.rfind('/')+1:])
        # Parse the query string
        parsed_query = parse_qs(unquote(row.uri_query))
        # Desktop and mobile have different formats
        if row.access_method == 'desktop':
            # Get the parameters called uri
            image_url = parsed_query.get("uri")
            if len(image_url)>0:
                image_name = unquote(image_url[0][image_url[0].rfind('/')+1:])
        else:
            # Get the titles parameter and then the string after File:
            file_name = parsed_query.get('titles')[0]
            image_title = file_name.split('File:')[1]
            image_name = unquote(image_title)
    except:
        # If there is no image return null
        image_name = None
    
    return Row(user_identifier=row.user_identifier, access_method=row.access_method, local_time=row.local_time,
                       title=title, image_name=image_name)

In [16]:
# The referer filter limits to image previews opened from Wikipedia articles
imageviews_query = """
    select CONCAT(MD5(CONCAT(CONCAT(client_ip, '{}'), access_method)), MD5(CONCAT(user_agent, '{}'))) as user_identifier,
        from_utc_timestamp(ts, geocoded_data['timezone']) as local_time,
        referer, uri_query, access_method,
        geocoded_data['country_code'] country_code
    from wmf.webrequest 
    where day = {}
    AND month = {}
    AND year = {}
    AND x_analytics_map['loggedIn'] is NULL
    AND agent_type = 'user'
    AND ts is not NULL
    AND (uri_host = 'en.wikipedia.org' or uri_host = 'en.m.wikipedia.org')
    AND referer LIKE '%wikipedia.org%'
    AND (
           (uri_path LIKE '%/beacon/media%' AND access_method = 'desktop')
        OR (uri_path = '/w/api.php' AND uri_query LIKE '%prop=imageinfo%' AND access_method = 'mobile web')
        )
"""

for day in daterange(start_date, end_date):
    imageviews = spark.sql(imageviews_query.format(salt1, salt2, day.day, day.month, day.year)).repartition(30)
    
    # Remove sensitive data
    imageviews_cleaned = without_sensitive_data(imageviews).drop("country_code")
    
    # Extract the titles from the urls and drop the original fields
    imageviews_with_titles = spark.createDataFrame(imageviews_cleaned.rdd.map(get_formatted_row))
    imageviews_with_titles.write.mode("overwrite")\
                .parquet("{}/imageviews/{}{:02d}{:02d}".format(DESTINATION_FOLDER, day.year, day.month, day.day))
    print("{}-{:02d}-{:02d} DONE".format(day.year, day.month, day.day))
    
print("Format:")
imageviews_with_titles

2021-01-03 DONE
2021-01-04 DONE
2021-01-05 DONE
2021-01-06 DONE
2021-01-07 DONE
2021-01-08 DONE
2021-01-09 DONE
2021-01-10 DONE
2021-01-11 DONE
2021-01-12 DONE
2021-01-13 DONE
2021-01-14 DONE
2021-01-15 DONE
2021-01-16 DONE
2021-01-17 DONE
2021-01-18 DONE
2021-01-19 DONE
2021-01-20 DONE
2021-01-21 DONE
2021-01-22 DONE
2021-01-23 DONE
2021-01-24 DONE
2021-01-25 DONE
2021-01-26 DONE
2021-01-27 DONE
2021-01-28 DONE
2021-01-29 DONE
2021-01-30 DONE
2021-01-31 DONE
Format:


DataFrame[access_method: string, image_name: string, local_time: timestamp, title: string, user_identifier: string]